In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd 
import numpy as np
import scipy
import matplotlib
import seaborn as sns
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
import re

Finally because the NN results were not satisfying I decided to check the results with the amino-acid content dataset enriched with the chemical properties of proteins. 

To achieve that protein properties from dataset (I) were combined with the preprocessed dataset (II)

In [5]:
proteins_with_functions = pd.read_csv("./Data/joined_tables.csv")

In [6]:
proteins_with_functions

,structureId,chainId,sequence,residueCount,len,A,C,D,E,F,...,T,U,V,W,Y,X,B,Z,classification,chainCount
0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,0.032468,...,0.032468,0.0,0.058442,0.012987,0.019481,0.0,0.0,0.0,TRANSPORT PROTEIN,1
1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,0.030303,...,0.072727,0.0,0.054545,0.018182,0.036364,0.0,0.0,0.0,HYDROLASE,1
2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,0.032468,0.0,0.051948,0.012987,0.019481,0.0,0.0,0.0,TRANSPORT PROTEIN,1
3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,0.029940,...,0.071856,0.0,0.053892,0.017964,0.035928,0.0,0.0,0.0,HYDROLASE,1
4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,0.032468,0.0,0.051948,0.012987,0.019481,0.0,0.0,0.0,TRANSPORT PROTEIN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322737,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,0.059278,...,0.041237,0.0,0.048969,0.015464,0.023196,0.0,0.0,0.0,ISOMERASE,1
322738,9XIM,A,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,393,0.117048,0.000000,0.083969,0.066158,0.055980,...,0.045802,0.0,0.050891,0.012723,0.030534,0.0,0.0,0.0,ISOMERASE,4
322739,9XIM,B,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,393,0.117048,0.000000,0.083969,0.066158,0.055980,...,0.045802,0.0,0.050891,0.012723,0.030534,0.0,0.0,0.0,ISOMERASE,4
322740,9XIM,C,SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...,1572,393,0.117048,0.000000,0.083969,0.066158,0.055980,...,0.045802,0.0,0.050891,0.012723,0.030534,0.0,0.0,0.0,ISOMERASE,4


In [7]:
proteins_data = pd.read_csv("./Data/pdb_data_no_dups.csv")

In [8]:
proteins_data

,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,1.78,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0
1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,NaN,NaN,2.00,38.45,NaN,NaN,1995.0
2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,NaN,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
3,102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0
4,102L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,165,1.74,18926.61,NaN,NaN,2.75,55.28,NaN,NaN,1993.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141396,9RUB,LYASE(CARBON-CARBON),X-RAY DIFFRACTION,Protein,932,2.60,101838.68,NaN,NaN,2.38,48.29,NaN,NaN,1991.0
141397,9TNA,T-RNA,X-RAY DIFFRACTION,NaN,0,NaN,24244.34,NaN,NaN,3.17,61.18,NaN,NaN,1986.0
141398,9WGA,LECTIN (AGGLUTININ),X-RAY DIFFRACTION,Protein,342,1.80,34270.22,NaN,NaN,2.50,50.76,NaN,NaN,1990.0
141399,9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,388,1.90,43542.29,NaN,NaN,2.79,55.93,NaN,NaN,1989.0


In [9]:
selected_columns= ["structureId","resolution","structureMolecularWeight","crystallizationTempK","densityMatthews","densityPercentSol","phValue"]

In [10]:
df = proteins_with_functions.merge(proteins_data[selected_columns].reset_index(), how="left", on="structureId")

In [11]:
df = df.drop_duplicates(subset="structureId",keep="first").reset_index()

In [12]:
df.drop(columns=["index","level_0"],inplace=True)

In [13]:
df

,structureId,chainId,sequence,residueCount,len,A,C,D,E,F,...,B,Z,classification,chainCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue
0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,0.032468,...,0.0,0.0,TRANSPORT PROTEIN,1,2.07,18112.80,NaN,3.09,60.20,9.0
1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,0.030303,...,0.0,0.0,HYDROLASE,1,1.74,18926.61,NaN,2.75,55.28,NaN
2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,0.0,0.0,TRANSPORT PROTEIN,1,1.84,18010.64,NaN,3.09,60.20,9.0
3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,0.029940,...,0.0,0.0,HYDROLASE,1,1.90,19092.72,NaN,2.70,54.46,NaN
4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,0.0,0.0,TRANSPORT PROTEIN,1,2.07,18093.78,NaN,3.09,60.30,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117508,9RNT,A,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPHKYNNYE...,104,104,0.067308,0.038462,0.057692,0.057692,0.038462,...,0.0,0.0,HYDROLASE,1,1.50,11134.77,NaN,2.09,41.23,NaN
117509,9RSA,A,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...,248,124,0.096774,0.064516,0.040323,0.040323,0.024194,...,0.0,0.0,HYDROLASE,2,1.80,27987.16,NaN,2.25,45.45,NaN
117510,9RUB,A,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...,932,466,0.133047,0.010730,0.062232,0.057940,0.049356,...,0.0,0.0,LYASE,2,2.60,101838.68,NaN,2.38,48.29,NaN
117511,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,0.059278,...,0.0,0.0,ISOMERASE,1,1.90,43542.29,NaN,2.79,55.93,NaN


Information about motifs (from notebook 3_motifs) was added in this new data frame

In [ ]:
import json

with open("./Data/beg_motifs_dict.json") as json_file:
    beg_motifs_dict = json.load(json_file)

with open("./Data/end_motifs_dict.json") as json_file:
    end_motifs_dict = json.load(json_file)

In [15]:
df["beg_motif"] = df["sequence"].apply(lambda x: x[:5]).map(beg_motifs_dict)
df["end_motif"] = df["sequence"].apply(lambda x: x[-5:]).map(end_motifs_dict)

In [16]:
df

,structureId,chainId,sequence,residueCount,len,A,C,D,E,F,...,classification,chainCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,beg_motif,end_motif
0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,0.032468,...,TRANSPORT PROTEIN,1,2.07,18112.80,NaN,3.09,60.20,9.0,26.0,NaN
1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,0.030303,...,HYDROLASE,1,1.74,18926.61,NaN,2.75,55.28,NaN,6.0,NaN
2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,TRANSPORT PROTEIN,1,1.84,18010.64,NaN,3.09,60.20,9.0,26.0,NaN
3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,0.029940,...,HYDROLASE,1,1.90,19092.72,NaN,2.70,54.46,NaN,6.0,NaN
4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,TRANSPORT PROTEIN,1,2.07,18093.78,NaN,3.09,60.30,9.0,26.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117508,9RNT,A,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPHKYNNYE...,104,104,0.067308,0.038462,0.057692,0.057692,0.038462,...,HYDROLASE,1,1.50,11134.77,NaN,2.09,41.23,NaN,NaN,NaN
117509,9RSA,A,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...,248,124,0.096774,0.064516,0.040323,0.040323,0.024194,...,HYDROLASE,2,1.80,27987.16,NaN,2.25,45.45,NaN,14.0,NaN
117510,9RUB,A,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...,932,466,0.133047,0.010730,0.062232,0.057940,0.049356,...,LYASE,2,2.60,101838.68,NaN,2.38,48.29,NaN,NaN,NaN
117511,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,0.059278,...,ISOMERASE,1,1.90,43542.29,NaN,2.79,55.93,NaN,NaN,NaN


Missing values were filled with mean and unidentified motifs with -1 

In [17]:
for column in df.columns[-8:-2]:
    df[column] = df[column].fillna(df.groupby('classification')[column].transform('mean'))

for column in df.columns[-2:]:
    df[column] = df[column].fillna(-1)
    df[column] = df[column].astype("int")

In [18]:
df

,structureId,chainId,sequence,residueCount,len,A,C,D,E,F,...,classification,chainCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,beg_motif,end_motif
0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,0.032468,...,TRANSPORT PROTEIN,1,2.07,18112.80,291.037497,3.09,60.20,9.000000,26,-1
1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,0.030303,...,HYDROLASE,1,1.74,18926.61,291.174328,2.75,55.28,6.662035,6,-1
2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,TRANSPORT PROTEIN,1,1.84,18010.64,291.037497,3.09,60.20,9.000000,26,-1
3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,0.029940,...,HYDROLASE,1,1.90,19092.72,291.174328,2.70,54.46,6.662035,6,-1
4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,TRANSPORT PROTEIN,1,2.07,18093.78,291.037497,3.09,60.30,9.000000,26,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117508,9RNT,A,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPHKYNNYE...,104,104,0.067308,0.038462,0.057692,0.057692,0.038462,...,HYDROLASE,1,1.50,11134.77,291.174328,2.09,41.23,6.662035,-1,-1
117509,9RSA,A,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...,248,124,0.096774,0.064516,0.040323,0.040323,0.024194,...,HYDROLASE,2,1.80,27987.16,291.174328,2.25,45.45,6.662035,14,-1
117510,9RUB,A,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...,932,466,0.133047,0.010730,0.062232,0.057940,0.049356,...,LYASE,2,2.60,101838.68,290.894503,2.38,48.29,7.082046,-1,-1
117511,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,0.059278,...,ISOMERASE,1,1.90,43542.29,291.368018,2.79,55.93,6.840613,-1,-1


In [19]:
proteins_dict = dict(zip(list(df["classification"].value_counts().index), range(len(df["classification"].value_counts().index.tolist()))))
df["proteins_encoded"] = df["classification"].map(proteins_dict)

The results were checked with Random Forest. 
The improvements were small compared to the tests from 2_sequence_based_model notebook, but overall this enriched dataset enabled to achieve the best results. Even though many of the experiments didn’t beat the baseline model they enable to explore the data better and observe the patterns in it.

In [48]:
from sklearn.ensemble import RandomForestClassifier

df_size = len(df)
train_size = int(len(df)*0.7)

proteins_cleaned_cleaned = df.sample(frac=1)

train = proteins_cleaned_cleaned[:train_size]
test = proteins_cleaned_cleaned[train_size:]

X_train = train.drop(columns=['structureId','chainId','sequence','classification',"proteins_encoded"])
y_train = train["proteins_encoded"]

X_test = test.drop(columns=['structureId','chainId','sequence','classification',"proteins_encoded"])
y_test = test["proteins_encoded"]

In [49]:
weights_dict = dict(zip(
                         range(len(train["classification"].value_counts().index.tolist())), 
                         train["classification"].value_counts().values.tolist()))

In [50]:
dict_weights = dict()

count_val = len(X_train)
for key, value in weights_dict.items():
    weight = 1-(value/count_val)
    dict_weights.update({key:weight})

In [51]:
model = RandomForestClassifier(class_weight=dict_weights)
model.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 0.7810695486208196,
                                     1: 0.8393853560096768,
                                     2: 0.8842922962836893,
                                     3: 0.9176381915656645,
                                     4: 0.9383410933758008,
                                     5: 0.9585334127572667,
                                     6: 0.9633839458296357,
                                     7: 0.9670795900752501,
                                     8: 0.9711399360556292,
                                     9: 0.9755285135972963,
                                     10: 0.9772790819241663,
                                     11: 0.9783610304039679,
                                     12: 0.9795037625062303,
                                     13: 0.983223720200829,
                                     14: 0.9876973948139414,
                                     15: 0.9882566041405804,
                                     16: 0.9882566041405804,
                                     17: 0.9889373807121409,
                                     18: 0.9894236496918271,
                                     19: 0.9929855699680278,
                                     20: 0.9931436073864258,
                                     21: 0.9935204658456825,
                                     22: 0.9948090786418508,
                                     23: 0.9951737803766153,
                                     24: 0.9958910271216523,
                                     25: 0.9962071019584483,
                                     26: 0.9962314154074327,
                                     27: 0.9967906247340717,
                                     28: 0.996900035254501,
                                     29: 0.9972525802647735, ...})

In [52]:
y_pred = model.predict(X_test)

Results improved compairing to the results of the sequence itself

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred, average='macro')}")
print(f"precision: {precision_score(y_test, y_pred, average='macro')}")
print(f"recall: {recall_score(y_test, y_pred, average='macro')}")

accuracy: 0.81803483292676
F1: 0.7777342850959699
precision: 0.8884393758719076
recall: 0.7055656340161425


In [ ]:
# df.to_csv("./Data/additional_features_dataset.csv")

In [21]:
df

,structureId,chainId,sequence,residueCount,len,A,C,D,E,F,...,chainCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,beg_motif,end_motif,proteins_encoded
0,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.110390,0.000000,0.038961,0.090909,0.032468,...,1,2.07,18112.80,291.037497,3.09,60.20,9.000000,26,-1,4
1,102L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,165,0.103030,0.000000,0.060606,0.048485,0.030303,...,1,1.74,18926.61,291.174328,2.75,55.28,6.662035,6,-1,0
2,102M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,1,1.84,18010.64,291.037497,3.09,60.20,9.000000,26,-1,4
3,103L,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,167,0.095808,0.000000,0.065868,0.047904,0.029940,...,1,1.90,19092.72,291.174328,2.70,54.46,6.662035,6,-1,0
4,103M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,154,0.116883,0.000000,0.038961,0.090909,0.038961,...,1,2.07,18093.78,291.037497,3.09,60.30,9.000000,26,-1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117508,9RNT,A,ACDYTCGSNCYSSSDVSTAQAAGYKLHEDGETVGSNSYPHKYNNYE...,104,104,0.067308,0.038462,0.057692,0.057692,0.038462,...,1,1.50,11134.77,291.174328,2.09,41.23,6.662035,-1,-1,0
117509,9RSA,A,KETAAAKFERQHMDSSTSAASSSNYCNQMMKSRNLTKDRCKPVNTF...,248,124,0.096774,0.064516,0.040323,0.040323,0.024194,...,2,1.80,27987.16,291.174328,2.25,45.45,6.662035,14,-1,0
117510,9RUB,A,MDQSSRYVNLALKEEDLIAGGEHVLCAYIMKPKAGYGYVATAAHFA...,932,466,0.133047,0.010730,0.062232,0.057940,0.049356,...,2,2.60,101838.68,290.894503,2.38,48.29,7.082046,-1,-1,5
117511,9XIA,A,MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...,388,388,0.121134,0.002577,0.095361,0.072165,0.059278,...,1,1.90,43542.29,291.368018,2.79,55.93,6.840613,-1,-1,9
